<a href="https://colab.research.google.com/github/Architrawat25/student_dropout_prediction/blob/main/student_dropout_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║   STUDENT DROPOUT PREDICTION — COMPLETE EXPERIMENTAL PIPELINE                ║
# ║                                                                              ║
# ║   Runs on: Google Colab (CPU or GPU)                                         ║
# ║   Produces: All tables, figures, and statistical test results                ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ══════════════════════════════════════════════════════════════════════════════
# CELL 1 — INSTALL DEPENDENCIES
# ══════════════════════════════════════════════════════════════════════════════
# Run this cell first. It installs all required libraries.
# Colab already has scikit-learn, numpy, pandas, matplotlib, seaborn pre-installed.

!pip install catboost optuna shap xgboost --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 38.3 MB/s eta 0:00:00


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 2 — IMPORTS AND GLOBAL CONFIGURATION
# ══════════════════════════════════════════════════════════════════════════════

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # non-interactive backend safe for Colab
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import warnings
import os
import json
import time
from copy import deepcopy

# ── Scikit-learn ──────────────────────────────────────────────────────────────
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve,
    confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.feature_selection import mutual_info_classif
from sklearn.pipeline import Pipeline

# ── Boosting ──────────────────────────────────────────────────────────────────
import xgboost as xgb
from catboost import CatBoostClassifier, Pool

# ── Hyperparameter Optimisation ───────────────────────────────────────────────
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)  # suppress verbose output

# ── SHAP ──────────────────────────────────────────────────────────────────────
import shap

# ── Statistical Tests ─────────────────────────────────────────────────────────
from scipy.stats import wilcoxon
from scipy import stats

# ── Suppress non-critical warnings ────────────────────────────────────────────
warnings.filterwarnings('ignore')

# ── Reproducibility ───────────────────────────────────────────────────────────
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# ── Output directory ─────────────────────────────────────────────────────────
OUTPUT_DIR = "paper_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ── Optuna trials per fold ────────────────────────────────────────────────────
# Increase to 100 for thorough tuning (~30 min total on CPU)
# 50 is a good balance (~15 min total on CPU)
N_OPTUNA_TRIALS = 50

print("✅ All imports successful.")
print(f"📁 Outputs will be saved to: ./{OUTPUT_DIR}/")
print(f"🔧 Optuna trials per fold: {N_OPTUNA_TRIALS}")

✅ All imports successful.
📁 Outputs will be saved to: ./paper_outputs/
🔧 Optuna trials per fold: 50


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 3 — LOAD AND INSPECT DATASET
# ══════════════════════════════════════════════════════════════════════════════
# ── INSTRUCTIONS ─────────────────────────────────────────────────────────────
# Upload your CSV file to Colab by running the cell below.
# It will open a file picker. Select your UCI dataset CSV file.
# The code auto-detects the delimiter (comma or semicolon).
# ─────────────────────────────────────────────────────────────────────────────

from google.colab import files

print("📂 Please upload your UCI dataset CSV file...")
uploaded = files.upload()

# Get the filename
csv_filename = list(uploaded.keys())[0]
print(f"✅ Uploaded: {csv_filename}")

# Auto-detect delimiter
with open(csv_filename, 'r') as f:
    first_line = f.readline()
delimiter = ';' if first_line.count(';') > first_line.count(',') else ','
print(f"🔍 Detected delimiter: '{delimiter}'")

df_raw = pd.read_csv(csv_filename, delimiter=delimiter)
print(f"\n📊 Dataset shape: {df_raw.shape}")
print(f"📋 Columns: {list(df_raw.columns)}")
print(f"\n🎯 Target variable distribution:")
print(df_raw['Target'].value_counts())

📂 Please upload your UCI dataset CSV file...


Saving data.csv to data (3).csv
✅ Uploaded: data (3).csv
🔍 Detected delimiter: ';'

📊 Dataset shape: (4424, 37)
📋 Columns: ['Marital status', 'Application mode', 'Application order', 'Course', 'Daytime/evening attendance\t', 'Previous qualification', 'Previous qualification (grade)', 'Nacionality', "Mother's qualification", "Father's qualification", "Mother's occupation", "Father's occupation", 'Admission grade', 'Displaced', 'Educational special needs', 'Debtor', 'Tuition fees up to date', 'Gender', 'Scholarship holder', 'Age at enrollment', 'International', 'Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)', 'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (gra

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 4 — DATA PREPARATION AND BINARIZATION
# ══════════════════════════════════════════════════════════════════════════════

# ── Step 1: Binarize target ───────────────────────────────────────────────────
# Dropout = 1 (positive class, at-risk)
# Graduate + Enrolled = 0 (negative class, non-at-risk)
df = df_raw.copy()
df['Target_Binary'] = df['Target'].apply(lambda x: 1 if str(x).strip() == 'Dropout' else 0)

print("🎯 Binarized target distribution:")
vc = df['Target_Binary'].value_counts()
print(f"  Non-Dropout (0): {vc[0]} ({vc[0]/len(df)*100:.1f}%)")
print(f"  Dropout     (1): {vc[1]} ({vc[1]/len(df)*100:.1f}%)")
print(f"  Class imbalance ratio: {vc[0]/vc[1]:.3f}")

# ── Step 2: Define feature sets ──────────────────────────────────────────────
# Identify all feature columns (everything except original target and binary target)
all_feature_cols = [c for c in df.columns if c not in ['Target', 'Target_Binary']]

# ── Macroeconomic feature names (exact UCI dataset column names) ──────────────
# These are the three columns we ablate in the ablation study
MACRO_COLS = [
    'Unemployment rate',
    'Inflation rate',
    'GDP'
]

# Verify macroeconomic columns exist
found_macro = [c for c in MACRO_COLS if c in all_feature_cols]
missing_macro = [c for c in MACRO_COLS if c not in all_feature_cols]

if missing_macro:
    print(f"\n⚠️  Macroeconomic columns not found with exact names: {missing_macro}")
    print("   Available columns containing 'rate', 'gdp', 'inflation', 'unemployment':")
    for c in all_feature_cols:
        if any(kw in c.lower() for kw in ['rate', 'gdp', 'inflation', 'unemploy']):
            print(f"   → '{c}'")
    print("\n   ⚠️  Please update MACRO_COLS above to match your dataset's exact column names.")
else:
    print(f"\n✅ Macroeconomic columns identified: {found_macro}")

# ── Step 3: Identify categorical columns for CatBoost ─────────────────────────
# In the UCI dropout dataset, these columns are encoded as integers
# but represent nominal categories
CATEGORICAL_COLS_CATBOOST = [
    'Marital status', 'Application mode', 'Application order', 'Course',
    'Daytime/evening attendance', 'Previous qualification',
    'Nacionality', "Mother's qualification", "Father's qualification",
    "Mother's occupation", "Father's occupation", 'Displaced',
    'Educational special needs', 'Debtor', 'Tuition fees up to date',
    'Gender', 'Scholarship holder', 'International'
]

# Keep only those that actually exist in the dataset
cat_cols_valid = [c for c in CATEGORICAL_COLS_CATBOOST if c in all_feature_cols]
cat_col_indices = [all_feature_cols.index(c) for c in cat_cols_valid]

print(f"\n📦 Total features: {len(all_feature_cols)}")
print(f"📦 Features in full model (Model A): {len(all_feature_cols)}")
print(f"📦 Features in ablated model (Model B): {len(all_feature_cols) - len(found_macro)}")

# ── Step 4: Prepare X, y ──────────────────────────────────────────────────────
X_full = df[all_feature_cols].values.astype(float)
X_ablated = df[[c for c in all_feature_cols if c not in found_macro]].values.astype(float)
y = df['Target_Binary'].values

feature_names_full = all_feature_cols
feature_names_ablated = [c for c in all_feature_cols if c not in found_macro]

# ── Step 5: Class weight ratio for boosting models ───────────────────────────
n_neg = np.sum(y == 0)
n_pos = np.sum(y == 1)
class_weight_ratio = n_neg / n_pos
print(f"\n⚖️  Class weight ratio (neg/pos) for boosting: {class_weight_ratio:.4f}")


🎯 Binarized target distribution:
  Non-Dropout (0): 3003 (67.9%)
  Dropout     (1): 1421 (32.1%)
  Class imbalance ratio: 2.113

✅ Macroeconomic columns identified: ['Unemployment rate', 'Inflation rate', 'GDP']

📦 Total features: 36
📦 Features in full model (Model A): 36
📦 Features in ablated model (Model B): 33

⚖️  Class weight ratio (neg/pos) for boosting: 2.1133


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 5 — HELPER FUNCTIONS
# ══════════════════════════════════════════════════════════════════════════════

def compute_metrics(y_true, y_pred, y_prob):
    """Compute all 5 evaluation metrics."""
    return {
        'accuracy':  accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall':    recall_score(y_true, y_pred, zero_division=0),
        'f1':        f1_score(y_true, y_pred, zero_division=0),
        'auc_roc':   roc_auc_score(y_true, y_prob)
    }


def summarize_cv_results(fold_metrics):
    """Compute mean ± std across folds for all metrics."""
    summary = {}
    metrics = fold_metrics[0].keys()
    for m in metrics:
        vals = [fold[m] for fold in fold_metrics]
        summary[m] = {'mean': np.mean(vals), 'std': np.std(vals), 'values': vals}
    return summary


def print_summary_table(name, summary):
    """Print a nicely formatted results table."""
    print(f"\n{'─'*65}")
    print(f"  {name}")
    print(f"{'─'*65}")
    print(f"  {'Metric':<15} {'Mean':>10} {'Std Dev':>10} {'Values (per fold)'}")
    print(f"{'─'*65}")
    for m, v in summary.items():
        vals_str = "  ".join([f"{x:.4f}" for x in v['values']])
        print(f"  {m:<15} {v['mean']:>10.4f} {v['std']:>10.4f}   [{vals_str}]")
    print(f"{'─'*65}")


def apply_mi_feature_selection(X_train, y_train, X_val, threshold_percentile=5):
    """
    Mutual information feature selection on training fold only.
    Removes features in the bottom `threshold_percentile` of MI scores.
    Returns selected X_train, X_val and selected feature indices.
    """
    mi_scores = mutual_info_classif(X_train, y_train, random_state=RANDOM_SEED)
    threshold = np.percentile(mi_scores, threshold_percentile)
    selected_idx = np.where(mi_scores >= threshold)[0]
    return X_train[:, selected_idx], X_val[:, selected_idx], selected_idx, mi_scores


# ── DeLong's Test Implementation ──────────────────────────────────────────────
def delong_roc_test(y_true, y_score1, y_score2):
    """
    DeLong's test to compare two correlated AUC-ROC values.
    Returns: z-statistic, p-value, AUC1, AUC2.
    Based on: DeLong et al. (1988) Biometrics 44(3):837-845.
    """
    def compute_midrank(x):
        J = np.argsort(x)
        Z = x[J]
        N = len(x)
        T = np.zeros(N, dtype=float)
        i = 0
        while i < N:
            j = i
            while j < N and Z[j] == Z[i]:
                j += 1
            T[i:j] = 0.5 * (i + j - 1)
            i = j
        T2 = np.empty(N, dtype=float)
        T2[J] = T + 1
        return T2

    def fastDeLong(predictions_sorted_transposed, label_1_count):
        m = label_1_count
        n = predictions_sorted_transposed.shape[1] - m
        positive_examples = predictions_sorted_transposed[:, :m]
        negative_examples = predictions_sorted_transposed[:, m:]
        k = predictions_sorted_transposed.shape[0]

        tx = np.empty([k, m], dtype=float)
        ty = np.empty([k, n], dtype=float)
        tz = np.empty([k, m + n], dtype=float)

        for r in range(k):
            tx[r, :] = compute_midrank(positive_examples[r, :])
            ty[r, :] = compute_midrank(negative_examples[r, :])
            tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])

        aucs = (tz[:, :m].sum(axis=1) - tx.sum(axis=1)) / (m * n)
        v01 = (tz[:, :m] - tx[:, :]) / n
        v10 = 1. - (tz[:, m:] - ty[:, :]) / m
        sx = np.cov(v01)
        sy = np.cov(v10)
        delongcov = sx / m + sy / n
        return aucs, delongcov

    y_true = np.asarray(y_true)
    y_score1 = np.asarray(y_score1)
    y_score2 = np.asarray(y_score2)

    sorted_indices = np.argsort(y_true)[::-1]
    y_true_sorted = y_true[sorted_indices]
    label_1_count = int(y_true_sorted.sum())

    predictions_sorted_transposed = np.vstack([
        y_score1[sorted_indices],
        y_score2[sorted_indices]
    ])

    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    auc1, auc2 = aucs[0], aucs[1]

    # Variance of (AUC1 - AUC2)
    var = delongcov[0, 0] + delongcov[1, 1] - 2 * delongcov[0, 1]
    if var <= 0:
        z_stat = 0.0
        p_value = 1.0
    else:
        z_stat = (auc1 - auc2) / np.sqrt(var)
        p_value = 2 * stats.norm.sf(abs(z_stat))

    return z_stat, p_value, auc1, auc2


print("✅ Helper functions defined.")


✅ Helper functions defined.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 6 — LOGISTIC REGRESSION WITH STRATIFIED 5-FOLD CV
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("  MODEL 1: LOGISTIC REGRESSION")
print("═"*65)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

lr_fold_metrics = []
lr_best_params_per_fold = []

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_full, y)):
    print(f"\n  Fold {fold_idx+1}/5 ...", end=" ")

    X_train_fold, X_val_fold = X_full[train_idx], X_full[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    # ── Mutual Information Feature Selection (on training fold only) ──────────
    X_train_sel, X_val_sel, sel_idx, mi_scores = apply_mi_feature_selection(
        X_train_fold, y_train_fold, X_val_fold
    )

    # ── Standard scaling ──────────────────────────────────────────────────────
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_sel)
    X_val_scaled   = scaler.transform(X_val_sel)

    # ── Grid Search for C ─────────────────────────────────────────────────────
    C_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
    lr_gs = GridSearchCV(
        LogisticRegression(
            class_weight='balanced',
            max_iter=2000,
            solver='lbfgs',
            random_state=RANDOM_SEED
        ),
        C_grid,
        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED),
        scoring='roc_auc',
        n_jobs=-1
    )
    lr_gs.fit(X_train_scaled, y_train_fold)
    best_lr = lr_gs.best_estimator_
    lr_best_params_per_fold.append(lr_gs.best_params_)

    # ── Predict ───────────────────────────────────────────────────────────────
    y_pred  = best_lr.predict(X_val_scaled)
    y_prob  = best_lr.predict_proba(X_val_scaled)[:, 1]

    metrics = compute_metrics(y_val_fold, y_pred, y_prob)
    lr_fold_metrics.append(metrics)
    print(f"AUC={metrics['auc_roc']:.4f}  F1={metrics['f1']:.4f}  Best C={lr_gs.best_params_['C']}")

lr_summary = summarize_cv_results(lr_fold_metrics)
print_summary_table("LOGISTIC REGRESSION — Cross-Validation Summary", lr_summary)



═════════════════════════════════════════════════════════════════
  MODEL 1: LOGISTIC REGRESSION
═════════════════════════════════════════════════════════════════

  Fold 1/5 ... AUC=0.9102  F1=0.8062  Best C=0.1

  Fold 2/5 ... AUC=0.9114  F1=0.7939  Best C=10

  Fold 3/5 ... AUC=0.9255  F1=0.7945  Best C=0.1

  Fold 4/5 ... AUC=0.9248  F1=0.7866  Best C=1

  Fold 5/5 ... AUC=0.9192  F1=0.8049  Best C=0.1

─────────────────────────────────────────────────────────────────
  LOGISTIC REGRESSION — Cross-Validation Summary
─────────────────────────────────────────────────────────────────
  Metric                Mean    Std Dev Values (per fold)
─────────────────────────────────────────────────────────────────
  accuracy            0.8660     0.0062   [0.8723  0.8633  0.8644  0.8565  0.8733]
  precision           0.7760     0.0155   [0.7886  0.7690  0.7733  0.7524  0.7966]
  recall              0.8198     0.0042   [0.8246  0.8204  0.8169  0.8239  0.8134]
  f1                  0.7972     0

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 7 — XGBOOST WITH OPTUNA TUNING
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("  MODEL 2: XGBOOST")
print("═"*65)

xgb_fold_metrics = []
xgb_best_params_per_fold = []

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_full, y)):
    print(f"\n  Fold {fold_idx+1}/5 — Optuna tuning ({N_OPTUNA_TRIALS} trials) ...", end=" ")

    X_train_fold, X_val_fold = X_full[train_idx], X_full[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    # ── MI Feature Selection ──────────────────────────────────────────────────
    X_train_sel, X_val_sel, sel_idx, _ = apply_mi_feature_selection(
        X_train_fold, y_train_fold, X_val_fold
    )

    # ── Inner split for Optuna ────────────────────────────────────────────────
    inner_skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    inner_splits = list(inner_skf.split(X_train_sel, y_train_fold))

    def xgb_objective(trial):
        params = {
            'n_estimators':      trial.suggest_int('n_estimators', 100, 1000),
            'max_depth':         trial.suggest_int('max_depth', 3, 10),
            'learning_rate':     trial.suggest_float('learning_rate', 0.01, 0.30, log=True),
            'subsample':         trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree':  trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_lambda':        trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
            'reg_alpha':         trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
            'scale_pos_weight':  class_weight_ratio,
            'use_label_encoder': False,
            'eval_metric':       'auc',
            'random_state':      RANDOM_SEED,
            'n_jobs':            -1
        }
        # Inner 5-fold CV for Optuna
        inner_aucs = []
        for tr_idx, vl_idx in inner_splits:
            Xtr, Xvl = X_train_sel[tr_idx], X_train_sel[vl_idx]
            ytr, yvl = y_train_fold[tr_idx], y_train_fold[vl_idx]
            model = xgb.XGBClassifier(**params)
            model.fit(Xtr, ytr, verbose=False)
            prob = model.predict_proba(Xvl)[:, 1]
            inner_aucs.append(roc_auc_score(yvl, prob))
        return np.mean(inner_aucs)

    study_xgb = optuna.create_study(direction='maximize',
                                     sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study_xgb.optimize(xgb_objective, n_trials=N_OPTUNA_TRIALS, show_progress_bar=False)

    best_p = study_xgb.best_params
    xgb_best_params_per_fold.append(best_p)

    # ── Train final model on full training fold ───────────────────────────────
    best_xgb = xgb.XGBClassifier(
        **best_p,
        scale_pos_weight=class_weight_ratio,
        use_label_encoder=False,
        eval_metric='auc',
        random_state=RANDOM_SEED,
        n_jobs=-1
    )
    best_xgb.fit(X_train_sel, y_train_fold)

    y_pred = best_xgb.predict(X_val_sel)
    y_prob = best_xgb.predict_proba(X_val_sel)[:, 1]

    metrics = compute_metrics(y_val_fold, y_pred, y_prob)
    xgb_fold_metrics.append(metrics)
    print(f"AUC={metrics['auc_roc']:.4f}  F1={metrics['f1']:.4f}  Best LR={best_p.get('learning_rate', 'N/A'):.4f}")

xgb_summary = summarize_cv_results(xgb_fold_metrics)
print_summary_table("XGBOOST — Cross-Validation Summary", xgb_summary)



═════════════════════════════════════════════════════════════════
  MODEL 2: XGBOOST
═════════════════════════════════════════════════════════════════

  Fold 1/5 — Optuna tuning (50 trials) ... AUC=0.9187  F1=0.7909  Best LR=0.0183

  Fold 2/5 — Optuna tuning (50 trials) ... AUC=0.9231  F1=0.7892  Best LR=0.0177

  Fold 3/5 — Optuna tuning (50 trials) ... AUC=0.9321  F1=0.7944  Best LR=0.0518

  Fold 4/5 — Optuna tuning (50 trials) ... AUC=0.9346  F1=0.8055  Best LR=0.0232

  Fold 5/5 — Optuna tuning (50 trials) ... AUC=0.9302  F1=0.8198  Best LR=0.0256

─────────────────────────────────────────────────────────────────
  XGBOOST — Cross-Validation Summary
─────────────────────────────────────────────────────────────────
  Metric                Mean    Std Dev Values (per fold)
─────────────────────────────────────────────────────────────────
  accuracy            0.8712     0.0072   [0.8644  0.8678  0.8667  0.8723  0.8846]
  precision           0.7981     0.0149   [0.7855  0.8081  0.

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 8 — CATBOOST MODEL A (FULL FEATURES) WITH OPTUNA TUNING
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("  MODEL 3: CATBOOST — MODEL A (Full Feature Set)")
print("═"*65)

cat_A_fold_metrics = []
cat_A_best_params_per_fold = []
cat_A_val_probs  = {}   # store per-fold probs for DeLong test
cat_A_val_labels = {}

# Get categorical feature indices that remain after MI selection
# We need to re-identify them after selection — handled per fold below

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_full, y)):
    print(f"\n  Fold {fold_idx+1}/5 — Optuna tuning ({N_OPTUNA_TRIALS} trials) ...", end=" ")

    X_train_fold, X_val_fold = X_full[train_idx], X_full[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    # ── MI Feature Selection ──────────────────────────────────────────────────
    X_train_sel, X_val_sel, sel_idx, _ = apply_mi_feature_selection(
        X_train_fold, y_train_fold, X_val_fold
    )

    # Re-map categorical column indices after feature selection
    # selected_features = [feature_names_full[i] for i in sel_idx]
    # cat_indices_fold = [selected_features.index(c) for c in cat_cols_valid
    #                     if c in selected_features]

    # ── Inner split ───────────────────────────────────────────────────────────
    inner_skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    inner_splits = list(inner_skf.split(X_train_sel, y_train_fold))

    def cat_objective(trial):
        params = {
            'iterations':        trial.suggest_int('iterations', 100, 1000),
            'depth':             trial.suggest_int('depth', 3, 10),
            'learning_rate':     trial.suggest_float('learning_rate', 0.01, 0.30, log=True),
            'l2_leaf_reg':       trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
            'random_strength':   trial.suggest_float('random_strength', 0.0, 1.0),
        }
        inner_aucs = []
        for tr_idx, vl_idx in inner_splits:
            Xtr, Xvl = X_train_sel[tr_idx], X_train_sel[vl_idx]
            ytr, yvl = y_train_fold[tr_idx], y_train_fold[vl_idx]
            w = np.where(ytr == 1, class_weight_ratio, 1.0)
            model = CatBoostClassifier(
                **params,
                class_weights={0: 1.0, 1: class_weight_ratio},
                random_seed=RANDOM_SEED,
                verbose=0,
                allow_writing_files=False
            )
            model.fit(Xtr, ytr)
            prob = model.predict_proba(Xvl)[:, 1]
            inner_aucs.append(roc_auc_score(yvl, prob))
        return np.mean(inner_aucs)

    study_cat_A = optuna.create_study(direction='maximize',
                                       sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study_cat_A.optimize(cat_objective, n_trials=N_OPTUNA_TRIALS, show_progress_bar=False)

    best_p = study_cat_A.best_params
    cat_A_best_params_per_fold.append(best_p)

    # ── Final model ───────────────────────────────────────────────────────────
    best_cat_A = CatBoostClassifier(
        **best_p,
        class_weights={0: 1.0, 1: class_weight_ratio},
        random_seed=RANDOM_SEED,
        verbose=0,
        allow_writing_files=False
    )
    best_cat_A.fit(X_train_sel, y_train_fold)

    y_pred = best_cat_A.predict(X_val_sel)
    y_prob = best_cat_A.predict_proba(X_val_sel)[:, 1]

    metrics = compute_metrics(y_val_fold, y_pred, y_prob)
    cat_A_fold_metrics.append(metrics)
    cat_A_val_probs[fold_idx]  = y_prob
    cat_A_val_labels[fold_idx] = y_val_fold

    print(f"AUC={metrics['auc_roc']:.4f}  F1={metrics['f1']:.4f}  Depth={best_p.get('depth')}")

cat_A_summary = summarize_cv_results(cat_A_fold_metrics)
print_summary_table("CATBOOST (Model A — Full Features) — Cross-Validation Summary", cat_A_summary)



═════════════════════════════════════════════════════════════════
  MODEL 3: CATBOOST — MODEL A (Full Feature Set)
═════════════════════════════════════════════════════════════════

  Fold 1/5 — Optuna tuning (50 trials) ... AUC=0.9134  F1=0.7896  Depth=6

  Fold 2/5 — Optuna tuning (50 trials) ... AUC=0.9254  F1=0.8063  Depth=6

  Fold 3/5 — Optuna tuning (50 trials) ... AUC=0.9360  F1=0.7958  Depth=5

  Fold 4/5 — Optuna tuning (50 trials) ... AUC=0.9306  F1=0.7951  Depth=7

  Fold 5/5 — Optuna tuning (50 trials) ... AUC=0.9308  F1=0.8211  Depth=5

─────────────────────────────────────────────────────────────────
  CATBOOST (Model A — Full Features) — Cross-Validation Summary
─────────────────────────────────────────────────────────────────
  Metric                Mean    Std Dev Values (per fold)
─────────────────────────────────────────────────────────────────
  accuracy            0.8716     0.0074   [0.8633  0.8746  0.8678  0.8678  0.8846]
  precision           0.7960     0.0123

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 9 — CATBOOST MODEL B (ABLATED — NO MACROECONOMIC FEATURES)
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("  MODEL 4: CATBOOST — MODEL B (Ablated: No Macroeconomic Features)")
print("═"*65)

cat_B_fold_metrics = []
cat_B_val_probs  = {}
cat_B_val_labels = {}

# Categorical column indices for the ablated feature set
cat_cols_ablated_valid = [c for c in cat_cols_valid if c not in found_macro]

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_ablated, y)):
    print(f"\n  Fold {fold_idx+1}/5 — Optuna tuning ({N_OPTUNA_TRIALS} trials) ...", end=" ")

    X_train_fold, X_val_fold = X_ablated[train_idx], X_ablated[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    # ── MI Feature Selection ──────────────────────────────────────────────────
    X_train_sel, X_val_sel, sel_idx, _ = apply_mi_feature_selection(
        X_train_fold, y_train_fold, X_val_fold
    )

    # selected_features = [feature_names_ablated[i] for i in sel_idx]
    # cat_indices_fold = [selected_features.index(c) for c in cat_cols_ablated_valid
    #                     if c in selected_features]

    # ── Inner split ───────────────────────────────────────────────────────────
    inner_skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    inner_splits = list(inner_skf.split(X_train_sel, y_train_fold))

    def cat_B_objective(trial):
        params = {
            'iterations':          trial.suggest_int('iterations', 100, 1000),
            'depth':               trial.suggest_int('depth', 3, 10),
            'learning_rate':       trial.suggest_float('learning_rate', 0.01, 0.30, log=True),
            'l2_leaf_reg':         trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
            'random_strength':     trial.suggest_float('random_strength', 0.0, 1.0),
        }
        inner_aucs = []
        for tr_idx, vl_idx in inner_splits:
            Xtr, Xvl = X_train_sel[tr_idx], X_train_sel[vl_idx]
            ytr, yvl = y_train_fold[tr_idx], y_train_fold[vl_idx]
            model = CatBoostClassifier(
                **params,
                class_weights={0: 1.0, 1: class_weight_ratio},
                random_seed=RANDOM_SEED,
                verbose=0,
                allow_writing_files=False
            )
            model.fit(Xtr, ytr)
            prob = model.predict_proba(Xvl)[:, 1]
            inner_aucs.append(roc_auc_score(yvl, prob))
        return np.mean(inner_aucs)

    study_cat_B = optuna.create_study(direction='maximize',
                                       sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study_cat_B.optimize(cat_B_objective, n_trials=N_OPTUNA_TRIALS, show_progress_bar=False)

    best_p = study_cat_B.best_params

    best_cat_B = CatBoostClassifier(
        **best_p,
        class_weights={0: 1.0, 1: class_weight_ratio},
        random_seed=RANDOM_SEED,
        verbose=0,
        allow_writing_files=False
    )
    best_cat_B.fit(X_train_sel, y_train_fold)

    y_pred = best_cat_B.predict(X_val_sel)
    y_prob = best_cat_B.predict_proba(X_val_sel)[:, 1]

    metrics = compute_metrics(y_val_fold, y_pred, y_prob)
    cat_B_fold_metrics.append(metrics)
    cat_B_val_probs[fold_idx]  = y_prob
    cat_B_val_labels[fold_idx] = y_val_fold

    print(f"AUC={metrics['auc_roc']:.4f}  F1={metrics['f1']:.4f}  Depth={best_p.get('depth')}")

cat_B_summary = summarize_cv_results(cat_B_fold_metrics)
print_summary_table("CATBOOST (Model B — Ablated) — Cross-Validation Summary", cat_B_summary)


═════════════════════════════════════════════════════════════════
  MODEL 4: CATBOOST — MODEL B (Ablated: No Macroeconomic Features)
═════════════════════════════════════════════════════════════════

  Fold 1/5 — Optuna tuning (50 trials) ... AUC=0.9117  F1=0.7774  Depth=8

  Fold 2/5 — Optuna tuning (50 trials) ... AUC=0.9217  F1=0.8035  Depth=6

  Fold 3/5 — Optuna tuning (50 trials) ... AUC=0.9342  F1=0.7965  Depth=6

  Fold 4/5 — Optuna tuning (50 trials) ... AUC=0.9305  F1=0.7931  Depth=4

  Fold 5/5 — Optuna tuning (50 trials) ... AUC=0.9264  F1=0.8092  Depth=5

─────────────────────────────────────────────────────────────────
  CATBOOST (Model B — Ablated) — Cross-Validation Summary
─────────────────────────────────────────────────────────────────
  Metric                Mean    Std Dev Values (per fold)
─────────────────────────────────────────────────────────────────
  accuracy            0.8680     0.0069   [0.8576  0.8723  0.8678  0.8644  0.8778]
  precision           0.790

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 10 — STATISTICAL SIGNIFICANCE TESTS
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("  STATISTICAL SIGNIFICANCE TESTS")
print("═"*65)

# ── AUC vectors per fold ──────────────────────────────────────────────────────
lr_aucs   = [m['auc_roc'] for m in lr_fold_metrics]
xgb_aucs  = [m['auc_roc'] for m in xgb_fold_metrics]
cat_A_aucs = [m['auc_roc'] for m in cat_A_fold_metrics]
cat_B_aucs = [m['auc_roc'] for m in cat_B_fold_metrics]

print(f"\n  Per-fold AUC-ROC values:")
print(f"  LR     : {[f'{v:.4f}' for v in lr_aucs]}")
print(f"  XGBoost: {[f'{v:.4f}' for v in xgb_aucs]}")
print(f"  Cat A  : {[f'{v:.4f}' for v in cat_A_aucs]}")
print(f"  Cat B  : {[f'{v:.4f}' for v in cat_B_aucs]}")

# ── Wilcoxon Signed-Rank Test ─────────────────────────────────────────────────
# NOTE: With n=5, minimum achievable two-tailed p-value is ~0.0625
# This is mathematically unavoidable and must be reported in the paper

def safe_wilcoxon(a, b, label):
    """Run Wilcoxon test with full reporting including power constraint note."""
    diffs = np.array(a) - np.array(b)
    n_pos = np.sum(diffs > 0)
    n_neg = np.sum(diffs < 0)
    n_tie = np.sum(diffs == 0)

    if np.all(diffs == 0):
        print(f"\n  {label}")
        print(f"  All differences are zero. Cannot perform Wilcoxon test.")
        return None, None, diffs

    try:
        stat, p = wilcoxon(a, b, alternative='two-sided')
    except ValueError as e:
        print(f"\n  {label}: {e}")
        return None, None, diffs

    direction = "A > B" if n_pos > n_neg else "B > A" if n_neg > n_pos else "Mixed"
    print(f"\n  {label}")
    print(f"  W = {stat:.1f}   p = {p:.4f}   Direction: {direction} "
          f"(+:{n_pos}, -:{n_neg}, ties:{n_tie})")
    print(f"  [Note: min. achievable p with n=5 is ≈0.0625]")
    if p < 0.05:
        print(f"  → REJECT null at α=0.05 ✅")
    elif p < 0.10:
        print(f"  → Borderline: consistent direction, limited by n=5 power constraint")
    else:
        print(f"  → FAIL TO REJECT null at α=0.05")
    return stat, p, diffs

print("\n── Wilcoxon Signed-Rank Tests (Pairwise AUC-ROC, n=5 folds) ──")

w1, p1, _ = safe_wilcoxon(cat_A_aucs, lr_aucs,   "CatBoost (Model A) vs. Logistic Regression")
w2, p2, _ = safe_wilcoxon(xgb_aucs,  lr_aucs,   "XGBoost vs. Logistic Regression")
w3, p3, _ = safe_wilcoxon(cat_A_aucs, xgb_aucs,  "CatBoost (Model A) vs. XGBoost")

# ── DeLong's Test — Ablation Study (Model A vs. Model B) ─────────────────────
print("\n── DeLong's Test (Ablation: Model A vs. Model B) ──")

# Aggregate all out-of-fold predictions for DeLong test
all_y_true_A, all_probs_A = [], []
all_y_true_B, all_probs_B = [], []

for fold_idx in range(5):
    all_y_true_A.extend(cat_A_val_labels[fold_idx])
    all_probs_A.extend(cat_A_val_probs[fold_idx])
    all_y_true_B.extend(cat_B_val_labels[fold_idx])
    all_probs_B.extend(cat_B_val_probs[fold_idx])

all_y_true_A = np.array(all_y_true_A)
all_probs_A  = np.array(all_probs_A)
all_probs_B  = np.array(all_probs_B)

z_delong, p_delong, auc_A_full, auc_B_full = delong_roc_test(
    all_y_true_A, all_probs_A, all_probs_B
)

print(f"\n  Model A (Full) AUC (DeLong):  {auc_A_full:.4f}")
print(f"  Model B (Ablated) AUC (DeLong): {auc_B_full:.4f}")
print(f"  ΔAUC = {auc_B_full - auc_A_full:.4f}")
print(f"  DeLong z-statistic = {z_delong:.4f}")
print(f"  p-value = {p_delong:.4f}")
if p_delong < 0.05:
    print(f"  → REJECT H₀₂ at α=0.05: Macroeconomic exclusion significantly degrades performance ✅")
else:
    print(f"  → FAIL TO REJECT H₀₂ at α=0.05: No statistically significant degradation")
    print(f"     ⇒ Empirical validation of limited marginal predictive contribution of")
    print(f"       macroeconomic indicators.")



═════════════════════════════════════════════════════════════════
  STATISTICAL SIGNIFICANCE TESTS
═════════════════════════════════════════════════════════════════

  Per-fold AUC-ROC values:
  LR     : ['0.9102', '0.9114', '0.9255', '0.9248', '0.9192']
  XGBoost: ['0.9187', '0.9231', '0.9321', '0.9346', '0.9302']
  Cat A  : ['0.9134', '0.9254', '0.9360', '0.9306', '0.9308']
  Cat B  : ['0.9117', '0.9217', '0.9342', '0.9305', '0.9264']

── Wilcoxon Signed-Rank Tests (Pairwise AUC-ROC, n=5 folds) ──

  CatBoost (Model A) vs. Logistic Regression
  W = 0.0   p = 0.0625   Direction: A > B (+:5, -:0, ties:0)
  [Note: min. achievable p with n=5 is ≈0.0625]
  → Borderline: consistent direction, limited by n=5 power constraint

  XGBoost vs. Logistic Regression
  W = 0.0   p = 0.0625   Direction: A > B (+:5, -:0, ties:0)
  [Note: min. achievable p with n=5 is ≈0.0625]
  → Borderline: consistent direction, limited by n=5 power constraint

  CatBoost (Model A) vs. XGBoost
  W = 6.0   p = 0.812

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 11 — CONFUSION MATRIX (OUT-OF-FOLD AGGREGATION)
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("  CONFUSION MATRIX — CatBoost (Out-of-Fold Aggregation)")
print("═"*65)

# Aggregate all OOF predictions (threshold = 0.50)
# This sums to exactly n=4,424 instances
oof_preds_A  = (all_probs_A >= 0.50).astype(int)
cm = confusion_matrix(all_y_true_A, oof_preds_A)

TN, FP, FN, TP = cm.ravel()
total = TN + FP + FN + TP

print(f"\n  Confusion Matrix (n = {total}):")
print(f"  {'':30s} Predicted Non-Dropout  Predicted Dropout")
print(f"  {'Actual Non-Dropout':30s}  TN = {TN:5d}          FP = {FP:5d}")
print(f"  {'Actual Dropout':30s}  FN = {FN:5d}          TP = {TP:5d}")
print(f"\n  Total instances: {total}  (should equal {len(y)})")

# Class-specific metrics
precision_dp = TP / (TP + FP) if (TP + FP) > 0 else 0
recall_dp    = TP / (TP + FN) if (TP + FN) > 0 else 0
fpr_dp       = FP / (FP + TN) if (FP + TN) > 0 else 0
fnr_dp       = FN / (FN + TP) if (FN + TP) > 0 else 0
accuracy_oof = (TP + TN) / total

print(f"\n  Dropout class (positive) metrics:")
print(f"    Precision    : {precision_dp:.4f} ({precision_dp*100:.1f}%)")
print(f"    Recall       : {recall_dp:.4f} ({recall_dp*100:.1f}%)")
print(f"    FPR          : {fpr_dp:.4f} ({fpr_dp*100:.1f}%)")
print(f"    FNR          : {fnr_dp:.4f} ({fnr_dp*100:.1f}%)")
print(f"    OOF Accuracy : {accuracy_oof:.4f} ({accuracy_oof*100:.1f}%)")



═════════════════════════════════════════════════════════════════
  CONFUSION MATRIX — CatBoost (Out-of-Fold Aggregation)
═════════════════════════════════════════════════════════════════

  Confusion Matrix (n = 4424):
                                 Predicted Non-Dropout  Predicted Dropout
  Actual Non-Dropout              TN =  2709          FP =   294
  Actual Dropout                  FN =   274          TP =  1147

  Total instances: 4424  (should equal 4424)

  Dropout class (positive) metrics:
    Precision    : 0.7960 (79.6%)
    Recall       : 0.8072 (80.7%)
    FPR          : 0.0979 (9.8%)
    FNR          : 0.1928 (19.3%)
    OOF Accuracy : 0.8716 (87.2%)


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 12 — SHAP ANALYSIS (CatBoost Model A, Final Fold)
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("  SHAP ANALYSIS — CatBoost (Model A, Final Fold Validation Set)")
print("═"*65)
print("  Training final CatBoost model for SHAP analysis...")

# Re-train CatBoost on fold 4 (index 4) training data for SHAP
fold_list = list(skf.split(X_full, y))
train_idx_final, val_idx_final = fold_list[4]

X_train_shap_fold = X_full[train_idx_final]
y_train_shap_fold = y[train_idx_final]
X_val_shap_fold   = X_full[val_idx_final]
y_val_shap_fold   = y[val_idx_final]

# Use best params from fold 4
best_p_shap = cat_A_best_params_per_fold[4]

# Re-apply MI selection on this fold
X_train_shap_sel, X_val_shap_sel, sel_idx_shap, mi_shap = apply_mi_feature_selection(
    X_train_shap_fold, y_train_shap_fold, X_val_shap_fold
)
selected_feature_names_shap = [feature_names_full[i] for i in sel_idx_shap]
cat_idx_shap = [selected_feature_names_shap.index(c) for c in cat_cols_valid
                if c in selected_feature_names_shap]

catboost_shap = CatBoostClassifier(
    **best_p_shap,
    # cat_features=cat_idx_shap,
    class_weights={0: 1.0, 1: class_weight_ratio},
    random_seed=RANDOM_SEED,
    verbose=0,
    allow_writing_files=False
)
catboost_shap.fit(X_train_shap_sel, y_train_shap_fold)
print("  ✅ CatBoost SHAP model trained.")

# ── Compute SHAP values ───────────────────────────────────────────────────────
print("  Computing SHAP values (this may take 1-2 minutes)...")
explainer = shap.TreeExplainer(catboost_shap)
shap_values = explainer.shap_values(X_val_shap_sel)

# For binary classification CatBoost, SHAP may return a list [class0, class1]
if isinstance(shap_values, list):
    shap_vals = shap_values[1]  # class 1 = Dropout
else:
    shap_vals = shap_values

print(f"  SHAP values shape: {shap_vals.shape}")

# ── Top 15 features by mean absolute SHAP ────────────────────────────────────
mean_abs_shap = np.abs(shap_vals).mean(axis=0)
shap_df = pd.DataFrame({
    'Feature': selected_feature_names_shap,
    'Mean_Abs_SHAP': mean_abs_shap
}).sort_values('Mean_Abs_SHAP', ascending=False).head(15)

print("\n  Top 15 Features by Mean |SHAP|:")
print(f"  {'Rank':<6} {'Feature':<45} {'Mean |SHAP|':>12}")
print(f"  {'─'*63}")
for rank, (_, row) in enumerate(shap_df.iterrows(), 1):
    print(f"  {rank:<6} {row['Feature']:<45} {row['Mean_Abs_SHAP']:>12.5f}")

# Save SHAP data
shap_df.to_csv(f"{OUTPUT_DIR}/shap_importance.csv", index=False)
print(f"\n  💾 SHAP importance saved to {OUTPUT_DIR}/shap_importance.csv")



═════════════════════════════════════════════════════════════════
  SHAP ANALYSIS — CatBoost (Model A, Final Fold Validation Set)
═════════════════════════════════════════════════════════════════
  Training final CatBoost model for SHAP analysis...
  ✅ CatBoost SHAP model trained.
  Computing SHAP values (this may take 1-2 minutes)...
  SHAP values shape: (884, 36)

  Top 15 Features by Mean |SHAP|:
  Rank   Feature                                        Mean |SHAP|
  ───────────────────────────────────────────────────────────────
  1      Curricular units 2nd sem (approved)                1.19879
  2      Tuition fees up to date                            0.53144
  3      Curricular units 1st sem (approved)                0.28195
  4      Course                                             0.27125
  5      Age at enrollment                                  0.26464
  6      Curricular units 2nd sem (grade)                   0.24187
  7      Gender                                       

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 13 — GENERATE ALL FIGURES
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("  GENERATING FIGURES")
print("═"*65)

plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight'
})

# ── FIGURE 1: ROC Curves (All 3 Models) ──────────────────────────────────────
print("\n  Generating Fig 1: ROC Curves...")

fig1, ax1 = plt.subplots(figsize=(7, 6))

models_roc = {
    'Logistic Regression': (lr_fold_metrics, 'steelblue',   '--'),
    'XGBoost':             (xgb_fold_metrics, 'darkorange', '-.' ),
    'CatBoost (Proposed)': (cat_A_fold_metrics,'darkgreen', '-'  ),
}

for name, (fold_metrics, color, ls) in models_roc.items():
    mean_auc = np.mean([m['auc_roc'] for m in fold_metrics])
    std_auc  = np.std([m['auc_roc'] for m in fold_metrics])

    # Compute mean ROC from OOF predictions
    # We need fold-specific predictions — use stored probs where available
    # For a clean ROC: use fold 4 predictions (representative fold)
    if name == 'Logistic Regression':
        fi = fold_list[4]
        X_tr, X_vl = X_full[fi[0]], X_full[fi[1]]
        y_tr, y_vl = y[fi[0]], y[fi[1]]
        Xtr_sel, Xvl_sel, _, _ = apply_mi_feature_selection(X_tr, y_tr, X_vl)
        scaler_roc = StandardScaler()
        Xtr_sc = scaler_roc.fit_transform(Xtr_sel)
        Xvl_sc = scaler_roc.transform(Xvl_sel)
        best_C  = lr_best_params_per_fold[4]['C']
        roc_model = LogisticRegression(C=best_C, class_weight='balanced',
                                        max_iter=2000, random_state=RANDOM_SEED)
        roc_model.fit(Xtr_sc, y_tr)
        prob_roc = roc_model.predict_proba(Xvl_sc)[:, 1]
        fpr_r, tpr_r, _ = roc_curve(y_vl, prob_roc)
    elif name == 'XGBoost':
        prob_roc = list(cat_A_val_probs.values())[4]  # placeholder — replace if stored
        # Recompute for fold 4
        fi = fold_list[4]
        X_tr, X_vl = X_full[fi[0]], X_full[fi[1]]
        y_tr, y_vl = y[fi[0]], y[fi[1]]
        Xtr_sel, Xvl_sel, _, _ = apply_mi_feature_selection(X_tr, y_tr, X_vl)
        bp = xgb_best_params_per_fold[4]
        roc_model = xgb.XGBClassifier(**bp, scale_pos_weight=class_weight_ratio,
                                       use_label_encoder=False, eval_metric='auc',
                                       random_state=RANDOM_SEED, n_jobs=-1)
        roc_model.fit(Xtr_sel, y_tr)
        prob_roc = roc_model.predict_proba(Xvl_sel)[:, 1]
        fpr_r, tpr_r, _ = roc_curve(y_vl, prob_roc)
    else:  # CatBoost
        prob_roc = cat_A_val_probs[4]
        fpr_r, tpr_r, _ = roc_curve(cat_A_val_labels[4], prob_roc)

    ax1.plot(fpr_r, tpr_r, color=color, linestyle=ls, lw=2,
             label=f"{name} (AUC = {mean_auc:.3f} ± {std_auc:.3f})")

ax1.plot([0, 1], [0, 1], 'k:', lw=1.2, label='Random Classifier (AUC = 0.500)')
ax1.fill_between([0, 1], [0, 1], alpha=0.04, color='gray')
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.01])
ax1.set_xlabel('False Positive Rate (1 − Specificity)')
ax1.set_ylabel('True Positive Rate (Sensitivity / Recall)')
ax1.set_title('Fig. 1. ROC Curves — All Models (Fold 5 Validation Set)')
ax1.legend(loc='lower right', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
plt.tight_layout()
fig1.savefig(f"{OUTPUT_DIR}/fig1_roc_curves.png")
plt.show()
print(f"  💾 Saved: {OUTPUT_DIR}/fig1_roc_curves.png")

# ── FIGURE 2: SHAP Summary Plot ───────────────────────────────────────────────
print("\n  Generating Fig 2: SHAP Summary Plot...")

fig2, ax2 = plt.subplots(figsize=(9, 7))
shap_plot_vals = shap_vals
shap_plot_data = X_val_shap_sel

# Get top 15 feature indices
top15_idx = shap_df.index.tolist()  # already sorted by importance
top15_names = shap_df['Feature'].tolist()

shap.summary_plot(
    shap_plot_vals[:, :len(selected_feature_names_shap)],
    shap_plot_data,
    feature_names=selected_feature_names_shap,
    max_display=15,
    show=False,
    plot_type="dot",
    color_bar_label="Feature Value"
)
plt.title("Fig. 2. SHAP Summary Plot — CatBoost (Top 15 Features)", fontsize=13, pad=12)
plt.tight_layout()
fig2.savefig(f"{OUTPUT_DIR}/fig2_shap_summary.png")
plt.show()
print(f"  💾 Saved: {OUTPUT_DIR}/fig2_shap_summary.png")

# ── FIGURE 3: SHAP Dependence Plot (Top Feature) ─────────────────────────────
print("\n  Generating Fig 3: SHAP Dependence Plot...")

top1_feat_name = shap_df.iloc[0]['Feature']
if top1_feat_name in selected_feature_names_shap:
    top1_feat_idx = selected_feature_names_shap.index(top1_feat_name)
    fig3, ax3 = plt.subplots(figsize=(8, 5))
    shap.dependence_plot(
        top1_feat_idx,
        shap_plot_vals,
        X_val_shap_sel,
        feature_names=selected_feature_names_shap,
        ax=ax3,
        show=False
    )
    ax3.set_title(f"Fig. 3. SHAP Dependence Plot — {top1_feat_name}", fontsize=13)
    ax3.set_xlabel(top1_feat_name)
    ax3.set_ylabel("SHAP Value (Impact on Dropout Probability)")
    ax3.grid(True, alpha=0.3)
    ax3.spines['top'].set_visible(False)
    ax3.spines['right'].set_visible(False)
    plt.tight_layout()
    fig3.savefig(f"{OUTPUT_DIR}/fig3_shap_dependence.png")
    plt.show()
    print(f"  💾 Saved: {OUTPUT_DIR}/fig3_shap_dependence.png")

# ── FIGURE 4: Confusion Matrix Heatmap ───────────────────────────────────────
print("\n  Generating Fig 4: Confusion Matrix...")

fig4, ax4 = plt.subplots(figsize=(6, 5))
cm_display = np.array([[TN, FP], [FN, TP]])
labels = [['TN\n{:,}'.format(TN), 'FP\n{:,}'.format(FP)],
          ['FN\n{:,}'.format(FN), 'TP\n{:,}'.format(TP)]]

sns.heatmap(
    cm_display,
    annot=np.array(labels), fmt='',
    cmap='Blues',
    xticklabels=['Predicted\nNon-Dropout', 'Predicted\nDropout'],
    yticklabels=['Actual\nNon-Dropout', 'Actual\nDropout'],
    linewidths=1, linecolor='white',
    cbar_kws={'label': 'Count'},
    ax=ax4
)
ax4.set_title(
    f"Fig. 4. Confusion Matrix — CatBoost (OOF Aggregation, n={total:,})\n"
    f"Precision={precision_dp:.3f}  Recall={recall_dp:.3f}  Accuracy={accuracy_oof:.3f}",
    fontsize=11, pad=10
)
plt.tight_layout()
fig4.savefig(f"{OUTPUT_DIR}/fig4_confusion_matrix.png")
plt.show()
print(f"  💾 Saved: {OUTPUT_DIR}/fig4_confusion_matrix.png")

print("\n  ✅ All figures saved successfully.")



═════════════════════════════════════════════════════════════════
  GENERATING FIGURES
═════════════════════════════════════════════════════════════════

  Generating Fig 1: ROC Curves...
  💾 Saved: paper_outputs/fig1_roc_curves.png

  Generating Fig 2: SHAP Summary Plot...
  💾 Saved: paper_outputs/fig2_shap_summary.png

  Generating Fig 3: SHAP Dependence Plot...
  💾 Saved: paper_outputs/fig3_shap_dependence.png

  Generating Fig 4: Confusion Matrix...
  💾 Saved: paper_outputs/fig4_confusion_matrix.png

  ✅ All figures saved successfully.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 14 — HYPERPARAMETER SUMMARY TABLE
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("  FINAL HYPERPARAMETER CONFIGURATIONS")
print("═"*65)

print("\n  Logistic Regression — Best C per fold:")
for i, p in enumerate(lr_best_params_per_fold):
    print(f"    Fold {i+1}: C = {p['C']}")

print("\n  XGBoost — Best params per fold:")
xgb_param_keys = ['n_estimators', 'max_depth', 'learning_rate', 'subsample',
                   'colsample_bytree', 'reg_lambda', 'reg_alpha']
for i, p in enumerate(xgb_best_params_per_fold):
    print(f"    Fold {i+1}: n_est={p.get('n_estimators')}  depth={p.get('max_depth')}  "
          f"lr={p.get('learning_rate', 0):.4f}  sub={p.get('subsample', 0):.2f}")

print("\n  CatBoost Model A — Best params per fold:")
for i, p in enumerate(cat_A_best_params_per_fold):
    print(f"    Fold {i+1}: iter={p.get('iterations')}  depth={p.get('depth')}  "
          f"lr={p.get('learning_rate', 0):.4f}  l2={p.get('l2_leaf_reg', 0):.4f}")


═════════════════════════════════════════════════════════════════
  FINAL HYPERPARAMETER CONFIGURATIONS
═════════════════════════════════════════════════════════════════

  Logistic Regression — Best C per fold:
    Fold 1: C = 0.1
    Fold 2: C = 10
    Fold 3: C = 0.1
    Fold 4: C = 1
    Fold 5: C = 0.1

  XGBoost — Best params per fold:
    Fold 1: n_est=951  depth=3  lr=0.0183  sub=0.94
    Fold 2: n_est=389  depth=9  lr=0.0177  sub=0.97
    Fold 3: n_est=399  depth=3  lr=0.0518  sub=0.94
    Fold 4: n_est=740  depth=3  lr=0.0232  sub=0.71
    Fold 5: n_est=506  depth=3  lr=0.0256  sub=0.88

  CatBoost Model A — Best params per fold:
    Fold 1: iter=763  depth=6  lr=0.0130  l2=0.3570
    Fold 2: iter=979  depth=6  lr=0.0105  l2=9.6398
    Fold 3: iter=865  depth=5  lr=0.0151  l2=2.0011
    Fold 4: iter=903  depth=7  lr=0.0128  l2=6.0948
    Fold 5: iter=628  depth=5  lr=0.0210  l2=3.3054


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 15 — COMPLETE RESULTS EXPORT FOR PAPER
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("  COMPLETE RESULTS — READY TO PASTE INTO PAPER TABLES")
print("═"*65)

def fmt(mean, std):
    return f"{mean:.3f} ± {std:.3f}"

metrics_order = ['accuracy', 'precision', 'recall', 'f1', 'auc_roc']
col_names     = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']

print("\n  ┌─ TABLE 2: Model Performance Comparison ─────────────────────────────────┐")
print(f"  │  {'Model':<35} {'Accuracy':>14} {'Precision':>14} {'Recall':>14} {'F1-Score':>14} {'AUC-ROC':>14} │")
print(f"  │  {'─'*35} {'─'*14} {'─'*14} {'─'*14} {'─'*14} {'─'*14} │")

for model_name, summary in [
    ("Logistic Regression (Baseline)", lr_summary),
    ("XGBoost",                        xgb_summary),
    ("CatBoost (Proposed)",            cat_A_summary),
]:
    row = "  │  " + f"{model_name:<35}"
    for m in metrics_order:
        row += f" {fmt(summary[m]['mean'], summary[m]['std']):>14}"
    row += " │"
    print(row)
print("  └─────────────────────────────────────────────────────────────────────────┘")

print("\n  ┌─ TABLE 3: Confusion Matrix (CatBoost OOF, n=4,424) ─┐")
print(f"  │  {'':30s} {'Pred: Non-Dropout':>20} {'Pred: Dropout':>20} {'Total':>10} │")
print(f"  │  {'Actual: Non-Dropout':30s} {'TN = '+str(TN):>20} {'FP = '+str(FP):>20} {TN+FP:>10} │")
print(f"  │  {'Actual: Dropout':30s} {'FN = '+str(FN):>20} {'TP = '+str(TP):>20} {FN+TP:>10} │")
print(f"  │  {'Total':30s} {TN+FN:>20} {FP+TP:>20} {total:>10} │")
print("  └─────────────────────────────────────────────────────┘")

print("\n  ┌─ TABLE 4: Ablation Study ─────────────────────────────────────────────────┐")
print(f"  │  {'Configuration':<45} {'Accuracy':>14} {'F1-Score':>14} {'AUC-ROC':>14} {'ΔAUC':>8} │")
print(f"  │  {'─'*45} {'─'*14} {'─'*14} {'─'*14} {'─'*8} │")

delta_auc = cat_B_summary['auc_roc']['mean'] - cat_A_summary['auc_roc']['mean']
for lbl, summ, d_str in [
    ("Model A: Full Feature Set (35 features)",               cat_A_summary, "—"),
    ("Model B: Without Macroeconomic Ind. (32 features)",     cat_B_summary, f"{delta_auc:+.3f}"),
]:
    row = f"  │  {lbl:<45}"
    for m in ['accuracy', 'f1', 'auc_roc']:
        row += f" {fmt(summ[m]['mean'], summ[m]['std']):>14}"
    row += f" {d_str:>8} │"
    print(row)
print("  └─────────────────────────────────────────────────────────────────────────┘")

print("\n  ┌─ TABLE 5: Statistical Significance Tests ────────────────────────────────┐")
print(f"  │  {'Comparison':<45} {'Test':>20} {'Statistic':>14} {'p-value':>10} │")
print(f"  │  {'─'*45} {'─'*20} {'─'*14} {'─'*10} │")

wilcoxon_results = [
    ("CatBoost vs. LR (AUC)",       "Wilcoxon S-R",  w1,        p1),
    ("XGBoost vs. LR (AUC)",        "Wilcoxon S-R",  w2,        p2),
    ("CatBoost vs. XGBoost (AUC)",  "Wilcoxon S-R",  w3,        p3),
    ("Model A vs. B (DeLong's)",    "DeLong's Test", z_delong,  p_delong),
]
for comp, test, stat, pval in wilcoxon_results:
    stat_str = f"{stat:.4f}" if stat is not None else "N/A"
    pval_str = f"{pval:.4f}" if pval is not None else "N/A"
    print(f"  │  {comp:<45} {test:>20} {stat_str:>14} {pval_str:>10} │")
print("  └─────────────────────────────────────────────────────────────────────────┘")

# ── Save all numerical results to JSON ───────────────────────────────────────
results_export = {
    "logistic_regression": {m: {"mean": lr_summary[m]['mean'], "std": lr_summary[m]['std'],
                                 "fold_values": lr_summary[m]['values']} for m in metrics_order},
    "xgboost":             {m: {"mean": xgb_summary[m]['mean'], "std": xgb_summary[m]['std'],
                                 "fold_values": xgb_summary[m]['values']} for m in metrics_order},
    "catboost_model_A":    {m: {"mean": cat_A_summary[m]['mean'], "std": cat_A_summary[m]['std'],
                                 "fold_values": cat_A_summary[m]['values']} for m in metrics_order},
    "catboost_model_B":    {m: {"mean": cat_B_summary[m]['mean'], "std": cat_B_summary[m]['std'],
                                 "fold_values": cat_B_summary[m]['values']} for m in metrics_order},
    "confusion_matrix":    {"TN": int(TN), "FP": int(FP), "FN": int(FN), "TP": int(TP),
                             "total": int(total), "precision": precision_dp,
                             "recall": recall_dp, "fpr": fpr_dp, "accuracy_oof": accuracy_oof},
    "statistical_tests": {
        "wilcoxon_catboost_vs_lr":   {"W": w1, "p": p1},
        "wilcoxon_xgboost_vs_lr":    {"W": w2, "p": p2},
        "wilcoxon_catboost_vs_xgb":  {"W": w3, "p": p3},
        "delong_modelA_vs_modelB":   {"z": z_delong, "p": p_delong,
                                       "auc_A": auc_A_full, "auc_B": auc_B_full,
                                       "delta_auc": float(auc_B_full - auc_A_full)},
    },
    "shap_top15": shap_df.to_dict(orient='records'),
    "hyperparameters": {
        "lr_best_C_per_fold":       lr_best_params_per_fold,
        "xgb_best_params_per_fold": xgb_best_params_per_fold,
        "catboost_A_best_per_fold": cat_A_best_params_per_fold,
    }
}

with open(f"{OUTPUT_DIR}/all_results.json", 'w') as f:
    json.dump(results_export, f, indent=2)

# Save summary CSVs
summary_rows = []
for model_name, summary in [
    ("Logistic Regression", lr_summary),
    ("XGBoost",             xgb_summary),
    ("CatBoost Model A",    cat_A_summary),
    ("CatBoost Model B",    cat_B_summary),
]:
    row = {"Model": model_name}
    for m in metrics_order:
        row[f"{m}_mean"] = round(summary[m]['mean'], 5)
        row[f"{m}_std"]  = round(summary[m]['std'],  5)
    summary_rows.append(row)
pd.DataFrame(summary_rows).to_csv(f"{OUTPUT_DIR}/model_performance.csv", index=False)

print(f"\n  💾 All results saved:")
print(f"     {OUTPUT_DIR}/all_results.json        ← Complete numerical results")
print(f"     {OUTPUT_DIR}/model_performance.csv   ← Table 2 data")
print(f"     {OUTPUT_DIR}/shap_importance.csv     ← SHAP rankings")
print(f"     {OUTPUT_DIR}/fig1_roc_curves.png     ← Fig 1")
print(f"     {OUTPUT_DIR}/fig2_shap_summary.png   ← Fig 2")
print(f"     {OUTPUT_DIR}/fig3_shap_dependence.png← Fig 3")
print(f"     {OUTPUT_DIR}/fig4_confusion_matrix.png← Fig 4")



═════════════════════════════════════════════════════════════════
  COMPLETE RESULTS — READY TO PASTE INTO PAPER TABLES
═════════════════════════════════════════════════════════════════

  ┌─ TABLE 2: Model Performance Comparison ─────────────────────────────────┐
  │  Model                                     Accuracy      Precision         Recall       F1-Score        AUC-ROC │
  │  ─────────────────────────────────── ────────────── ────────────── ────────────── ────────────── ────────────── │
  │  Logistic Regression (Baseline)       0.866 ± 0.006  0.776 ± 0.015  0.820 ± 0.004  0.797 ± 0.007  0.918 ± 0.006 │
  │  XGBoost                              0.871 ± 0.007  0.798 ± 0.015  0.802 ± 0.018  0.800 ± 0.011  0.928 ± 0.006 │
  │  CatBoost (Proposed)                  0.872 ± 0.007  0.796 ± 0.012  0.807 ± 0.010  0.802 ± 0.011  0.927 ± 0.008 │
  └─────────────────────────────────────────────────────────────────────────┘

  ┌─ TABLE 3: Confusion Matrix (CatBoost OOF, n=4,424) ─┐
  │    

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 16 — DOWNLOAD ALL OUTPUTS AS ZIP
# ══════════════════════════════════════════════════════════════════════════════

import shutil

zip_name = "paper_outputs"
shutil.make_archive(zip_name, 'zip', OUTPUT_DIR)
files.download(f"{zip_name}.zip")

print(f"\n✅ PIPELINE COMPLETE.")
print(f"📦 Download started: {zip_name}.zip")
print(f"\n📋 NEXT STEPS:")
print(f"   1. Open paper_outputs/all_results.json")
print(f"   2. Replace all placeholder numbers in your paper with the real values")
print(f"   3. Insert the 4 saved figures into your Word document")
print(f"   4. Generate Figures 5 and 6 (System Architecture, Preprocessing Flowchart)")
print(f"      using draw.io, Lucidchart, or PowerPoint — these cannot be auto-generated")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ PIPELINE COMPLETE.
📦 Download started: paper_outputs.zip

📋 NEXT STEPS:
   1. Open paper_outputs/all_results.json
   2. Replace all placeholder numbers in your paper with the real values
   3. Insert the 4 saved figures into your Word document
   4. Generate Figures 5 and 6 (System Architecture, Preprocessing Flowchart)
      using draw.io, Lucidchart, or PowerPoint — these cannot be auto-generated
